In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf

import scipy.optimize as sco
import scipy.interpolate as sci

import simu as sim

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor


In [ ]:
path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
file_name = "\V4.5_1MPE.xlsx"

In [ ]:
hw, par, cond = fe.initialize(path, file_name)
tabl, res, PL, testings = modf.PL_fsolve(par, cond)

In [ ]:
import model_fsolve_v2 as modf2
dic_cond = {"alpha":1, "eta":0.001, "rho":1000, "Qin_c":0, "Qin_d":2.5e-5}
cond2 = modf2.condition(dic_cond)

In [ ]:
tabl2, Xsol = modf2.PL_fsolve(par, cond2)
tabl, tabl2

In [ ]:
tabl, res, PL, testings, fun = modf.PL_fsolve(par, cond, series=[])

In [ ]:
# Qmax = 2.5e-5
# list_Qin = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.])*Qmax
# list_alpha = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, .7, .8, .9, 1.])
# df_testings = fe.testing_series_Qmax(Qmax, list_Qin, list_alpha, par, cond)
# df_testings.to_excel(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification" + file_name[:-5]+"_test.xlsx", index=False)

In [ ]:
import seaborn as sns
df_testings = pd.read_excel(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification" + file_name[:-5]+"_test.xlsx")
# plots df_testings.plot(x='alpha', y = 'Pin collect') colored by Qin
df_testings['DPin'] = df_testings['Pout distrib'] - df_testings['Pin distrib']
sns.relplot(data=df_testings, x='alpha', y='DPin', hue='Qin distrib', aspect=1.61)
# plt.plot(list_alpha, [Pin(2.5e-5, alpha) for alpha in list_alpha])
plt.show()

In [ ]:
def MATRIX(df_testings):
    Qin_list = df_testings['Qin distrib'].unique()
    df_coefficients = pd.DataFrame(columns=['Qin distrib', 'a_in', 'b_in', 'c_in', 'a_out', 'b_out', 'c_out', 'a_x', 'b_x', 'c_x'])
    for Qin in Qin_list:
        mask = df_testings['Qin distrib'] == Qin
        alpha = np.array(df_testings[mask]['alpha'])  
        DPin = np.array(df_testings[mask]['DPin']) 
        DPout = np.array(df_testings[mask]['Pin collect'])
        DPx = np.array(df_testings[mask]['Pout distrib'])
        coefficients_in = np.polyfit(alpha, DPin, 2)
        a_in = coefficients_in[0]
        b_in = coefficients_in[1]
        c_in = coefficients_in[2]
        coefficients_out = np.polyfit(alpha, DPout, 2)
        a_out = coefficients_out[0]
        b_out = coefficients_out[1]
        c_out = coefficients_out[2]
        coefficients_x = np.polyfit(alpha, DPx, 2)
        a_x = coefficients_x[0]
        b_x = coefficients_x[1]
        c_x = coefficients_x[2]
        df_coefficients.loc[len(df_coefficients)] = [Qin, a_in, b_in, c_in, a_out, b_out, c_out, a_x, b_x, c_x]

    a_in = np.array(df_coefficients['a_in']) 
    b_in = np.array(df_coefficients['b_in'])
    c_in = np.array(df_coefficients['c_in'])
    a_out = np.array(df_coefficients['a_out'])
    b_out = np.array(df_coefficients['b_out'])
    c_out = np.array(df_coefficients['c_out'])
    a_x = np.array(df_coefficients['a_x'])
    b_x = np.array(df_coefficients['b_x'])
    c_x = np.array(df_coefficients['c_x'])

    def M(Qin_list, a, b, c):
        matrix = pd.DataFrame(columns=['A', 'B', 'C'])
        COEFF = np.polyfit(Qin_list, a, 2)
        matrix.loc[len(matrix)] = [COEFF[0], COEFF[1], COEFF[2]]
        COEFF = np.polyfit(Qin_list, b, 2)
        matrix.loc[len(matrix)] = [COEFF[0], COEFF[1], COEFF[2]]
        COEFF = np.polyfit(Qin_list, c, 2)
        matrix.loc[len(matrix)] = [COEFF[0], COEFF[1], COEFF[2]]
        return matrix

    MATRIX_in = M(Qin_list, a_in, b_in, c_in)
    MATRIX_out = M(Qin_list, a_out, b_out, c_out)
    MATRIX_x = M(Qin_list, a_x, b_x, c_x)

    return MATRIX_in, MATRIX_out, MATRIX_x

MATRIX_in, MATRIX_out, MATRIX_x = MATRIX(df_testings)

def DPin(Qin, alpha):
    coeffs = np.matmul(MATRIX_in, np.array([Qin**2, Qin, 1]))
    return np.matmul(coeffs, np.array([alpha**2, alpha, 1]))

def DPout(Qin, alpha):
    coeffs = np.matmul(MATRIX_out, np.array([Qin**2, Qin, 1]))
    return np.matmul(coeffs, np.array([alpha**2, alpha, 1]))

def DPx(Qin, alpha):
    coeffs = np.matmul(MATRIX_x, np.array([Qin**2, Qin, 1]))
    return np.matmul(coeffs, np.array([alpha**2, alpha, 1]))

In [ ]:
DPx(2.5e-5, 1.)

In [ ]:
Pin_c = []
Pin_d = []
Pout_d = []
for i in range(len(df_testings)):
    Qin = df_testings['Qin distrib'][i]
    alpha = df_testings['alpha'][i]
    Pin_c.append(DPout(Qin, alpha))
    Pin_d.append(DPx(Qin, alpha) - DPin(Qin, alpha))
    Pout_d.append(DPx(Qin, alpha))


df_testings['Pin distrib try'] = Pin_d
df_testings['Pout distrib try'] = Pout_d
df_testings['Pin collect try'] = Pin_c

(abs(df_testings['Pout distrib try'] - df_testings['Pout distrib'])*100/df_testings['Pout distrib']).max()

In [ ]:
df_testings[df_testings['Qin distrib']==2.5e-5]['Pin distrib'] - df_testings[df_testings['Qin distrib']==2.5e-5]['Pout distrib'] 

In [ ]:
df_testings[df_testings['alpha']==0.5].plot(x='Qin distrib', y = 'Pin distrib')
# plt.plot([0,2.5e-5],[0,2440*0.5])
plt.show()

In [ ]:
df_testings[df_testings['alpha']==0.5]

In [ ]:
tabl, res, PL, testings, fun = modf.PL_fsolve(par, cond)

In [ ]:
tabl

In [ ]:
list_QF = np.array([0, 50, 100, 150, 200, 250, 300, 350])/3600000
list_QF_out = np.array([0, 50, 100, 150, 200, 250, 300])/3600000
list_alpha = np.array([0.1, 0.3, 0.5, 0.7, 0.9])
fe.testing_series(path+file_name, list_QF, list_QF_out, list_alpha, par, cond)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

df_testings = pd.read_excel(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification\MICOE_risers_x6_testings.xlsx").drop(columns=['Unnamed: 0'])
X = df_testings[['QF', 'QF_out', 'alpha']].to_numpy()
yin = df_testings['DPin'].to_numpy()
yout = df_testings['DPout'].to_numpy()
yx = df_testings['DPx'].to_numpy()

### Cette func fait un split aléatoire des datas : le même pour les 3 modèles. random
X_train, X_test, yin_train, yin_test, yout_train, yout_test, yx_train, yx_test = train_test_split(X, yin, yout, yx, test_size=0.2)

model_in = RandomForestRegressor() 
model_in.fit(X_train, yin_train)

model_out = RandomForestRegressor() 
model_out.fit(X_train, yout_train)

model_x = RandomForestRegressor() 
model_x.fit(X_train, yx_train)

yin_pred = model_in.predict(X_test)
yout_pred = model_out.predict(X_test)
yx_pred = model_x.predict(X_test)

r2_in = r2_score(yin_test, yin_pred)
r2_out = r2_score(yout_test, yout_pred)
r2_x = r2_score(yx_test, yx_pred)


print(f"R² du modèle 'DPin': {r2_in :.5f}")
print(f"R² du modèle 'DPout': {r2_out :.5f}")
print(f"R² du modèle 'DPx': {r2_x :.5f}")

i = 0
print(model_in.predict([[list_QF[i], list_QF_out[0], list_alpha[i]]])[0]/model_in.predict([[list_QF[i], list_QF_out[-1], list_alpha[i]]])[0])
print(model_x.predict([[list_QF[i], list_QF_out[0], list_alpha[i]]])[0]/model_x.predict([[list_QF[i], list_QF_out[-1], list_alpha[i]]])[0])
print(model_out.predict([[list_QF[i], list_QF_out[0], list_alpha[i]]])[0]/model_out.predict([[list_QF[i], list_QF_out[-1], list_alpha[i]]])[0])


In [ ]:
hw, par, cond = fe.initialize(path, "\MICOE_simplified.xlsx")
tabl, res, df_PL, testings, f = modf.PL_fsolve(par, cond, simplified=True)
f

In [ ]:
tabl

In [ ]:
hw, par, cond = fe.initialize(path, "\MICOE.xlsx")
tabl, res, df_PL, testings = modf.PL_fsolve(par, cond)
tabl

In [ ]:
tabl, res, df_PL, testings = modf.PL_fsolve(par, cond)

In [ ]:
list_PL_U, list_tabl_U = modf.PL_fsolve_range(par, cond, np.array([50, 200, 350, 500, 650])/3600000)

In [ ]:
list_PL_Z, list_tabl_Z = modf.PL_fsolve_range(par, cond, np.array([50, 200, 350, 500, 650])/3600000)

In [ ]:
### plot les pertes de charges dans l'échangeur list_PL_U et list_PL_Z en fonction du débit en L/h Vdot = [50, 200, 350, 500, 650]

plt.figure()
plt.plot(np.array([50, 200, 350, 500, 650]), list_PL_U, label="PL_U")
plt.plot(np.array([50, 200, 350, 500, 650]), list_PL_Z, label="PL_Z")
plt.xlabel("Vdot [L/h]")
plt.ylabel("Pertes de charge [bar]")
plt.legend()
plt.show()


In [ ]:
dplt.solve_plot(path, file_name)

In [ ]:
cond["Dv"] = 0.165e-3
cond["Vdot"] = cond["Dv"]*3600000
tabl, res, df_PL, testings, df_Re = modf.PL_fsolve(par, cond)

In [ ]:
df_Re.to_excel(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Re_0.165.xlsx")

In [ ]:
list_Dv = [0.027e-3, 0.055e-3, 0.110e-3, 0.165e-3, 0.220e-3]
list_Vdot = np.array(list_Dv)*3600000
list_PL_CFD = np.array([1067, 1279, 3861, 5072, 9573])
list_PL_model ,list_tabl = modf.PL_fsolve_range(par, cond, list_Dv, 1.1/4)
list_ratio = list_PL_CFD/list_PL_model
plt.plot(list_Vdot, list_ratio)
# plot une ligne égale à 1 en pointillé
plt.plot([list_Vdot[0], list_Vdot[-1]], [1, 1], 'k--')
plt.xlabel('Vdot (L/h)')
plt.ylabel('ratio PL_CFD/PL_model')
plt.show()

In [ ]:
cond['Dv'] = 0.055e-3 
cond['Vdot'] = cond['Dv']*3600000 
tabl, res, df_PL, t, f = modf.PL_fsolve(par, cond)

In [ ]:
df = pd.read_csv(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\CFD\V4.5_MPEx4\processed\0.055_exhaust_pressure.csv")
last = df["Position"].max() - 0.07
df["riser"] = [(df.loc[i]["Position"])*len(tabl)/last for i in range(len(df))]
df[df["Position"]<last].plot(x="riser", y="Pressure")
tabl["Pout"].plot(x=tabl.index)
point2 = df[df["Position"]<last].iloc[-1]["Pressure"]
df2=df[df["Position"]<last]

In [ ]:
df = pd.read_csv(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\CFD\V4.5_MPEx4\processed\0.055_intake_pressure.csv")
last = df["Position"].max()
df["riser"] = [(df.loc[i]["Position"] - 0.074)*len(tabl)/(last- 0.074) for i in range(len(df))]
df[df["Position"]>0.074].plot(x="riser", y="Pressure")
tabl["Pin"].plot(x=tabl.index)
point1 = df[df["Position"]>0.074].iloc[0]["Pressure"]
df1=df[df["Position"]>0.074]

In [ ]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)
df1-df2

In [ ]:
D = '0.055'
df1 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD/V4.5_MPEx4/processed/" +D+ "_risers_entry_pressure_per_riser.csv")
df2 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD\V4.5_MPEx4/processed/"+D+"_risers_entry+0.050_pressure_per_riser.csv")
df3 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD/V4.5_MPEx4/processed/"+D+"_risers_after_first_bend_pressure_per_riser.csv")
df4 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD/V4.5_MPEx4/processed/"+D+"_risers_before_bj_pressure_per_riser.csv")   
df5 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD/V4.5_MPEx4/processed/"+D+"_risers_before_exhaust_pressure_per_riser.csv")

(df1["Pressure"]-df5["Pressure"]).plot(x="Group")
df_PL["RPL riser"].plot()

In [ ]:
df3

In [ ]:
PL = point1 - point2
PL

In [ ]:
((df1-df2)['Pressure']* (67/5)/(df3-df4)['Pressure']).mean()

In [ ]:
entry_5cm = np.array([0.7396875 - 0.7404128, 0.003751432 - 0.004154148, -0.3049988 + 0.5200261])
entry = np.array([0.7785593-0.7787191, 0.03597014-0.03598411, -0.5199889+0.3049711])
entry_5cm/entry

In [ ]:
hx, par, cond = fe.initialize(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs",r"\V4.5_1MPE.xlsx")
list_QF = np.array([50,200,350,500,650,800])/3600000
list_QF_out = np.array([50,200,350,500])/3600000
list_alpha = [0.01, 0.05, 0.1, 0.4, 0.7, 1]
df = fe.testing_series(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification\V4.5_1MPE_testings.xlsx", list_QF, list_QF_out, list_alpha, par, cond)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

df_testings = pd.read_excel("G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification\V4.5_1MPE_testings.xlsx")
X = df_testings[['QF', 'QF_out', 'alpha']].to_numpy()
yin = df_testings['DPin'].to_numpy()
yout = df_testings['DPout'].to_numpy()
yx = df_testings['DPx'].to_numpy()

model_in = RandomForestRegressor() 
model_in.fit(X, yin)

model_out = RandomForestRegressor() 
model_out.fit(X, yout)

model_x = RandomForestRegressor() 
model_x.fit(X, yx)

df_testings['DPin_test'] = model_in.predict(X)
df_testings['DPout_test'] = model_out.predict(X)
df_testings['DPx_test'] = model_x.predict(X)

In [ ]:
dplt.K_abaque(df_testings, 'DPout')

In [ ]:
hx, par, cond = fe.initialize(path, file_name)
list_Vdot = np.array([150, 300, 450, 600])
list_Dv = list_Vdot/3600000
for coef in [1, 1.1, 1.2, 1.4] :
    list_PL, list_tabl = modf.PL_fsolve_range(par, cond, list_Dv, 0.25*coef)
    plt.plot(list_Vdot, list_PL/1000, label = f'f *= {coef}')
plt.legend()
plt.show()
    


In [ ]:
df2 = pd.DataFrame(modf.calc(df['qx'], par, cond)).transpose()
df2.columns = ["Qin", "Qout", "uin", "ux", "uout", "Rein", "Rex", "Reout", "fin", "fx", "fout", "Kx_in", "Ky_in", "Kx_out", "Ky_out", "K_se", "Lex"]
df = df.join(df2)
df = df.join(df_PL)


In [ ]:
path = 'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Simulation_outputs'
path_inputs = 'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs'
name = '\V4.41.xlsx'
list_Vdot = np.array([150, 300, 450, 600])
hx, par, cond = fe.initialize(path_inputs, name)

writer = pd.ExcelWriter(path+name, engine = 'openpyxl')
for Vdot in list_Vdot :
    cond['Vdot'] = Vdot
    cond['Dv'] = Vdot/3600000
    df, PL, df_PL, testings = modf.PL_fsolve(par, cond)
    df2 = pd.DataFrame(modf.calc(df['qx'], par, cond)).transpose()
    df2.columns = ["Qin", "Qout", "uin", "ux", "uout", "Rein", "Rex", "Reout", "fin", "fx", "fout", "Kx_in", "Ky_in", "Kx_out", "Ky_out", "K_se", "Lex"]
    df = df.join(df2)
    df = df.join(df_PL)
    df.to_excel(writer, sheet_name= f"Vdot={Vdot}", index=False)

writer.close()


In [ ]:
dplt.solve_plot(path, file_name)

In [ ]:
df_K = pd.read_excel("G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification\V4.5_1MPE_testings.xlsx")

In [ ]:
df_A = df_K[['QF', 'QF_out','alpha']]
df_A['constant']=1
df_A['sqrt'] = np.sqrt(df_A['QF'])
df_A['QF*alpha'] = df_A['alpha']*df_A['QF']
df_A['sqrt_alpha'] = df_A['alpha']*np.sqrt(df_A['QF'])
df_A['QF_out*alpha'] = df_A['alpha']*df_A['QF_out']
df_A['alpha2'] = df_A['alpha']**2
df_A['QF*alpha2'] = df_A['alpha2']*df_A['QF']
df_A['sqrt_alpha2'] = df_A['alpha2']*np.sqrt(df_A['QF'])
df_A['QF_out*alpha2'] = df_A['alpha2']*df_A['QF_out']
df_A = df_A[['constant', 'QF', 'sqrt', 'QF_out', 'alpha', 'QF*alpha', 'QF_out*alpha', 'sqrt_alpha', 'alpha2', 'QF*alpha2','QF_out*alpha2', 'sqrt_alpha2']]

In [ ]:
A = df_A.to_numpy()
B = df_K[['Kyin']].to_numpy()
X = np.linalg.lstsq(A,B)[0]

err = (A.dot(X)-B)*100/B
np.abs(err.round()).mean()

In [ ]:
X.transpose()*(df_A.mean().to_numpy())

In [ ]:
df = pd.DataFrame(X.transpose())
df.to_excel("G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs\V4.5_1MPE_Kyin.xlsx", index=False)

In [ ]:
df_K['Kyin_test'] = A.dot(X)

In [ ]:
dplt.K_abaque(df_K,'Kyin')

In [ ]:
TUV = pd.read_excel('G:\Drive partagés\Cercle Hard\Innovation\Hydraulique\PL_database\\230727_TUV_V4.41_x1_Z_water.xlsx')
path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
file_name = r"\V4.41.xlsx"
hx, par, cond = fe.initialize(path, file_name)

In [ ]:
list_Vdot = np.array(list(TUV['Vdot']))[::2]
list_Dv = list_Vdot/3600000
f = 20
list_PL,list_tabl = modf.PL_fsolve_range(par, cond, list_Dv, f)
plt.plot(list_Vdot, list_PL/1000, label = f'fapp = {f}')
plt.plot(list(TUV['Vdot']), list(TUV['PL']), label = 'TÜV')
plt.legend()
plt.show()



In [ ]:
# read excel file
excel_file = pd.ExcelFile(r'G:\Drive partagés\Cercle Hard\Innovation\Hydraulique\components\230725_PL_V4.41.xlsx')

# create dictionary with dataframes
df_dict = {}
for sheet_name in excel_file.sheet_names:
    df_dict[sheet_name] = excel_file.parse(sheet_name)
    df_dict[sheet_name].drop(columns='Unnamed: 0',inplace=True)

In [ ]:
for i in range(4):
    key = list(df_dict.keys())[i]
    list_Vdot = np.array(df_dict[key]['Vdot_per_panel'])
    list_Dv = list_Vdot*(i+1)/3600000

    path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
    file_name = f"\V4.41_x{i+1}.xlsx"
    hx, par, cond = fe.initialize(path, file_name)
    list_PL, list_tabl = modf.PL_fsolve_range(par, cond, list_Dv[1::39], 20)

    plt.plot(list_Vdot[1::39], list_PL/1000, label='Simulation')
    plt.plot(list_Vdot, np.array(df_dict[key]['PL']), label='Experiments')
    plt.xlabel('Vdot par panneau [L/h]')
    plt.ylabel('PL [kPa]')
    plt.legend()
    plt.show()



In [ ]:
list(df_dict.keys())

In [ ]:
tabl, res, PL, testings, fun = modf.PL_fsolve(par, cond)

In [ ]:
res

In [ ]:
path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
file_name = "\V4.41_x2.xlsx"
hw, par_41, cond = fe.initialize(path, file_name)
file_name = "\V4.6_x2.xlsx"
hw, par_6, cond = fe.initialize(path, file_name)


In [ ]:
path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
file_name = "\V4.41.xlsx"
hw, par_41, cond = fe.initialize(path, file_name)
file_name = "\V4.6.xlsx"
hw, par_6, cond = fe.initialize(path, file_name)

list_res_41_1 = []
list_res_6_1 = []

for Vdot in [25, 50, 75, 100, 150, 200, 400, 600]:
    cond['Vdot'] = Vdot
    cond['Dv'] = Vdot/3600000
    tabl, res, PL, testings, fun = modf.PL_fsolve(par_41, cond)
    list_res_41_1.append(res)
    tabl, res, PL, testings, fun = modf.PL_fsolve(par_6, cond)
    list_res_6_1.append(res)

plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_41_1, label='V4.41 x1')
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_6_1, label='V4.6 x1')
plt.xlabel('Vdot [L/h]')
plt.ylabel('Pertes de charge [bar]')
plt.legend()
plt.show()

In [ ]:
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], [list_res_41_1[i]/list_res_6_1[i] for i in range(len(list_res_41_1))], label='PL ratio V4.41/V4.6 : x1')
plt.xlabel('Vdot [L/h]')
plt.ylabel('PL ratio')
plt.legend()
plt.show()

In [ ]:
path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
file_name = "\V4.41_x2.xlsx"
hw, par_41, cond = fe.initialize(path, file_name)
file_name = "\V4.6_x2.xlsx"
hw, par_6, cond = fe.initialize(path, file_name)

list_res_41_2 = []
list_res_6_2 = []

for Vdot in [25, 50, 75, 100, 150, 200]:
    cond['Vdot'] = Vdot
    cond['Dv'] = Vdot/3600000
    tabl, res, PL, testings, fun = modf.PL_fsolve(par_41, cond)
    list_res_41_2.append(res)
    tabl, res, PL, testings, fun = modf.PL_fsolve(par_6, cond)
    list_res_6_2.append(res)

plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_41_2, label='V4.41 x2')
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_6_2, label='V4.6 x2')
plt.xlabel('Vdot [L/h]')
plt.ylabel('Pertes de charge [bar]')
plt.legend()
plt.show()

In [ ]:
for Vdot in [400, 600]:
    cond['Vdot'] = Vdot
    cond['Dv'] = Vdot/3600000
    tabl, res, PL, testings, fun = modf.PL_fsolve(par_41, cond)
    list_res_41_2.append(res)
    tabl, res, PL, testings, fun = modf.PL_fsolve(par_6, cond)
    list_res_6_2.append(res)

In [ ]:
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_41_2, label='V4.41 x2')
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_6_2, label='V4.6 x2')
plt.xlabel('Vdot [L/h]')
plt.ylabel('Pertes de charge [bar]')
plt.legend()
plt.show()

In [ ]:
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], [list_res_41_2[i]/list_res_6_2[i] for i in range(len(list_res_41_1))], label='PL ratio V4.41/V4.6 : x2')
plt.xlabel('Vdot [L/h]')
plt.ylabel('PL ratio')
plt.legend()
plt.show()

In [ ]:
path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
file_name = "\V4.41_x3.xlsx"
hw, par_41, cond = fe.initialize(path, file_name)
file_name = "\V4.6_x3.xlsx"
hw, par_6, cond = fe.initialize(path, file_name)

list_res_41_3 = []
list_res_6_3 = []

for Vdot in [25, 50, 75, 100, 150, 200, 400, 600]:
    cond['Vdot'] = Vdot
    cond['Dv'] = Vdot/3600000
    tabl, res, PL, testings, fun = modf.PL_fsolve(par_41, cond)
    list_res_41_3.append(res)
    tabl, res, PL, testings, fun = modf.PL_fsolve(par_6, cond)
    list_res_6_3.append(res)

plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_41_3, label='V4.41 x3')
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_6_3, label='V4.6 x3')
plt.xlabel('Vdot [L/h]')
plt.ylabel('Pertes de charge [bar]')
plt.legend()
plt.show()

In [ ]:
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_41_3, label='V4.41 x3')
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], list_res_6_3, label='V4.6 x3')
plt.xlabel('Vdot [L/h]')
plt.ylabel('Pertes de charge [bar]')
plt.legend()
plt.show()

In [ ]:
plt.plot([25, 50, 75, 100, 150, 200, 400, 600], [list_res_41_3[i]/list_res_6_3[i] for i in range(len(list_res_41_3))], label='PL ratio V4.41/V4.6 : x3')
plt.xlabel('Vdot [L/h]')
plt.ylabel('PL ratio')
plt.legend()
plt.show()

In [ ]:
import os

DP_export = []
for Vdot in [10, 30, 60, 100, 150]:
    df = pd.read_csv(f'G:\Drive partagés\Cercle Hard\Innovation\Hydraulique\PL_1_riser\processed\\line_{Vdot}.csv')
    DP_export.append(df['Static Pressure'].iloc[-1]-df['Static Pressure'].iloc[0])
    plt.plot(df['Position'], df['Static Pressure'], label=f'Vdot = {Vdot} L/h')

plt.xlabel('Position [m]')
plt.ylabel('Pressure [Pa]')
plt.legend()
plt.show()

In [ ]:
DP_list = [-0.744-13.85, 0.0645 +4.6, -0.07+71.6, 122.9+0.18, 188+0.36]
Vdot_list = [10, 30, 60, 100, 150]
mdot_list= np.array(Vdot_list)*1e3/3600000/304
plt.plot(mdot_list, DP_list, label='method = surface integral')
#plt.plot(mdot_list, DP_export, label='method = line-to-export')
plt.xlabel('mdot [kg/s]')
plt.ylabel('DP [Pa]')
plt.show()